# Assignment 2: Iterative-Deepening Search

Bradley Pospeck

## Overview

The goal of this assignment is to implement iterative deepening search. Two primary functions will be implemented in order to accomplish a depth limited search. 

The algorithm will be tested primarily on 2 puzzles. First, will be the 8 puzzle. The 8 puzzle is a simple 3x3 grid. One space on the grid is a blank, while the rest are filled with tiles numbered from 1-8. Generally, the goal would be to organize the tiles such that they're ordered from left to right, top to bottom. For this assignment, the goal can be any formation and the algorithm will attempt to find a way to that goal state from an arbitrary starting state.

The second puzzle is personal choice. I implement a maze. Initially, I implemented a specific size maze for simplicity, but then go on to generalize it to any nxn maze. In both cases, the maze is randomly generated. The depth limited search will try to find a way from the starting position to the end position.

## Imports

In [1]:
import copy
import random
import math

## Iterative Deepening Functions

The following functions are implemented for this assignment:

  * `iterativeDeepeningSearch(startState, goalState, actionsF, takeActionF, maxDepth)`
  * `depthLimitedSearch(state, goalState, actionsF, takeActionF, depthLimit)`
  
`depthLimitedSearch` is called by `iterativeDeepeningSearch` with `depthLimit`s of $0, 1, \ldots, $ `maxDepth`. Both must return either the solution path as a list of states, or the strings `cutoff` or `failure`.  `failure` signifies that all states were searched and the goal was not found. `cutoff` indicates that the maxDepth given to `iterativeDeepeningSearch` was not enough to fully explore all possible states and actions. This means that there may or may not be a solution path if it was `cutoff`.

Both functions receive the arguments

  * the starting state, 
  * a function `actionsF` that is given a state and returns a list of valid actions from that state,
  * a function `takeActionF` that is given a state and an action and returns the new state that results from applying the action to the state,
  * the goal state,
  * either a `depthLimit` for `depthLimitedSearch`, or `maxDepth` for `iterativeDeepeningSearch`.

In [2]:
def depthLimitedSearch(state, goalState, actionsF, takeActionF, depthLimit):
    """Recursive definition of a depth limited search. Attempts to find the 'goalState' from the 'state'. Uses passed in 
    functions 'actionsF' and 'takeActionF' to determine which actions are viable before iterating through those actions."""
    if state == goalState:
        return []
    if depthLimit == 0:
        return 'cutoff'
    cutoffOccurred = False
    for action in actionsF(state):
        childState = takeActionF(state, action)
        result = depthLimitedSearch(childState, goalState, actionsF, takeActionF, depthLimit-1)
        if result == 'cutoff':
            cutoffOccurred = True
        elif result != 'failure':
            result.insert(0,childState)
            return result
    if cutoffOccurred:
        return 'cutoff'
    else:
        return 'failure'

In [3]:
def iterativeDeepeningSearch(startState, goalState, actionsF, takeActionF, maxDepth):
    """Calls 'depthLimitedSearch' at a range of depths up to 'maxDepth' in order to try and find the 'goalState' from
    'startState'"""
    for depth in range(maxDepth):
        result = depthLimitedSearch(startState, goalState, actionsF, takeActionF, depth)
        if result == 'failure':
            return 'failure'
        if result != 'cutoff':
            result.insert(0,startState)       
            return result
    return 'cutoff'

In order to test out the functions, 2 different puzzles will be used. The first will be the 8 puzzle. The second will be a puzzle of my choice. I decided to use a maze.

## 8 Puzzle Implementation

#### 8 Puzzle Functions

The state of the 8 puzzle will simply be a list of integers. The blank space is represented by a 0.

Required functions, as per the assignment, for the 8-puzzle are the following.

  * `findBlank_8p(state)`: return the row and column index for the location of the blank (the 0 value).
  * `actionsF_8p(state)`: returns a list of up to four valid actions that can be applied in `state`. Return them in the order `left`, `right`, `up`, `down`, though only if each one is a valid action.
  * `takeActionF_8p(state, action)`: return the state that results from applying `action` in `state`.
  * `goalTestF_8p(state, goalState)`: return `True` if state is a goal state.
  * `printPath_8p(startState, goalState, path)`: print a solution path in a readable form.  You choose the format.

In [4]:
def findBlank_8p(state):
    """Finds and returns the (row,column) index for the current state's blank/0 location"""
    index = state.index(0)
    row = int(index/3)
    col = index % 3
    return (row, col)

In [5]:
def actionsF_8p(state):
    """Takes the current state and returns a list of the valid actions. Four possible actions: left, right, up, down."""
    actions = ['left','right','up','down']
    blank = findBlank_8p(state)
    if(blank[0] == 0):
        if 'up' in actions: actions.remove('up')#actions should never be removed before here, but it's good housekeeping to check
    if(blank[0] == 2):
        if 'down' in actions: actions.remove('down')
    if(blank[1] == 0):
        if 'left' in actions: actions.remove('left')
    if(blank[1] == 2):
        if 'right' in actions: actions.remove('right')
    return actions

In [6]:
def takeActionF_8p(state, action):
    """Apply the given action to state and return the resulting state"""
    cp = copy.copy(state) # If a copy isn't taken, the action will permanently change the original state passed in, which is bad
    index = cp.index(0)
    if(action == 'left'):
        temp = cp[index-1]
        cp[index-1] = 0
        cp[index] = temp
    elif(action == 'right'):
        temp = cp[index+1]
        cp[index+1] = 0
        cp[index] = temp
    elif(action == 'up'):
        temp = cp[index-3]
        cp[index-3] = 0
        cp[index] = temp
    else: #action is down
        temp = cp[index+3]
        cp[index+3] = 0
        cp[index] = temp
    return cp

In [7]:
def printPath_8p(startState, goalState, path):
    """Prints out the startState and goalState of the 8 puzzle. Prints out the path between both states, if it exists."""
    if path != 'failure' and path != 'cutoff':
        print('Solution path to the goal is {} steps long\n'.format(len(path)-1))
    print('Starting state:')
    printState_8p(startState)
    print('Goal state:')
    printState_8p(goalState)
    print('Solution path:')
    if path == 'failure':
        print('Failed: No solution found')
    elif path == 'cutoff':
        print('Path was cutoff')
    else:
        for i in path:
            printState_8p(i)

In [8]:
def printState_8p(state):
    """Prints out the 3x3 state of the 8 puzzle"""
    index = state.index(0)
    state[index] = ' '
    print('{} {} {}'.format(state[0],state[1],state[2]))
    print('{} {} {}'.format(state[3],state[4],state[5]))
    print('{} {} {}'.format(state[6],state[7],state[8]))
    print()
    state[index] = 0

#### 8 Puzzle Testing

To make testing easier/faster, I'll use random state generators. One was provided with the assignment, `randomStartState`, but I wrote my own `randomState` function for completely random states. For the sake of notebook brevity, I'll only show one successful example of each case: finding a path, cutoff, and failure.

In [9]:
def randomState():
    """Generates a completely random state for 8 puzzle"""
    state = []
    rand = [0,1,2,3,4,5,6,7,8]
    for i in range(9):
        select = random.choice(rand)
        state.insert(i, select)
        rand.remove(select)
    return state

In [10]:
def randomStartState(goalState, actionsF, takeActionF, nSteps):
    """Generates a startState given a random set of actions from a desired goalState"""
    state = goalState
    for i in range(nSteps):
        state = takeActionF(state, random.choice(actionsF(state)))
    return state

I'll start with start/goal pairings that are guaranteed to have a goal to make sure it works properly when there is a path. **Note:** When printing out the 8 puzzle, the 0's were removed in favor of actual blank spaces in order see where the blank is quicker.

In [11]:
goal = randomState()
start = randomStartState(goal, actionsF_8p, takeActionF_8p, 20)
path = iterativeDeepeningSearch(start, goal, actionsF_8p, takeActionF_8p, 10)
printPath_8p(start, goal, path)

Solution path to the goal is 8 steps long

Starting state:
7 4  
6 8 2
5 1 3

Goal state:
7 2 8
6 4 3
  5 1

Solution path:
7 4  
6 8 2
5 1 3

7 4 2
6 8  
5 1 3

7 4 2
6   8
5 1 3

7   2
6 4 8
5 1 3

7 2  
6 4 8
5 1 3

7 2 8
6 4  
5 1 3

7 2 8
6 4 3
5 1  

7 2 8
6 4 3
5   1

7 2 8
6 4 3
  5 1



Now I'll make start/goal pairings competely random to see if cutoffs are handled correctly without crashing. **Note:** It's entirely possible that the 2 random states won't end up yielding a cutoff path. So when running the below cell, keep in mind a solution path may still be found.

In [12]:
goal = randomState()
start = randomState()
path = iterativeDeepeningSearch(start, goal, actionsF_8p, takeActionF_8p, 5)
printPath_8p(start, goal, path)

Starting state:
2 7 1
3 4 6
  5 8

Goal state:
3 6 7
  4 1
2 5 8

Solution path:
Path was cutoff


At the very least, testing my `printPath_8p` function with a failure is easy enough to do.

In [13]:
printPath_8p(start, goal, 'failure')

Starting state:
2 7 1
3 4 6
  5 8

Goal state:
3 6 7
  4 1
2 5 8

Solution path:
Failed: No solution found


## 5x5 Maze Implementation

I decided it would be fun to try and setup randomly generated mazes for my second puzzle on this assignment. It's a simple maze with only left, right, up, and down actions. As with the 8 puzzle, the state of the maze will be a list, but of symbols this time.

#### Maze Functions

For simplicity, I hardcoded values to work with a randomly generated 7x7 maze. However, the entire perimeter of the maze is always a wall. Everything within that wall will be randomly generated. This effectively makes the maze a 5x5. Traversable paths are denoted by a space, while walls are denoted by a star. A single 'S' will be placed at a random location to start the maze. To generate a goal, the starting maze will be passed in to `goalMaze`. This function will automatically move the 'S' to any traversable spot, not a wall. There is a chance it will simply place the 'S' in the starting location too.

In [14]:
def mazeActionsF(state):
    """Takes the current maze state and returns a list of the valid actions. Four possible actions: left, right, up, down."""
    actions = ['left','right','up','down']
    s = state.index('S')
    if state[s-1] == '*':
        if 'left' in actions: actions.remove('left') # simple insurance check, even though actions shouldn't be removed before
    if state[s+1] == '*':
        if 'right' in actions: actions.remove('right')
    if state[s-7] == '*':
        if 'up' in actions: actions.remove('up')
    if state[s+7] == '*':
        if 'down' in actions: actions.remove('down')
    return actions

In [15]:
def takeMazeActionF(state, action):
    """Apply the given action to the maze state and return the resulting state"""
    cp = copy.copy(state) # If a copy isn't taken, the action will permanently change the original state passed in, which is bad
    index = cp.index('S')
    if(action == 'left'):
        temp = cp[index-1]
        cp[index-1] = 'S'
        cp[index] = temp
    elif(action == 'right'):
        temp = cp[index+1]
        cp[index+1] = 'S'
        cp[index] = temp
    elif(action == 'up'):
        temp = cp[index-7]
        cp[index-7] = 'S'
        cp[index] = temp
    else: #action is down
        temp = cp[index+7]
        cp[index+7] = 'S'
        cp[index] = temp
    return cp

In [16]:
def randomMaze():
    """Randomly generates a 5x5 maze with a starting position, 'S'. The full size is a 7x7, but the entire perimeter is a wall"""
    maze = []
    space = ['*','*','*',' ',' ',' ',' ',' ',' ',' ','S'] # 3 walls, 7 spaces so walls aren't randomly picked as often. 1 start
    for i in range(49): # 7x7 maze, with walls on all edges; effectively a 5x5 randomly generated maze
        if i<7 or i>41 or i%7==0 or i%7==6:
            maze.append('*')
        else:
            place = random.choice(space)
            if place == 'S':
                space.remove(place)
            if 'S' not in maze and 'S' in space and i == 40: # Forces a start location if it wasn't randomly picked before here
                maze.append('S')
                space.remove('S')
                continue
            maze.append(place)
    return maze

In [17]:
def mazeGoal(maze):
    """Takes an existing maze as an argument, and shifts the 'S' to an open space to generate and return a goal maze"""
    goalMaze = copy.copy(maze)
    start = goalMaze.index('S')
    goalMaze[start] = ' '
    available = []
    for i in range(len(goalMaze)):
        if goalMaze[i] == ' ':
            available.append(i)
    goal = random.choice(available)
    goalMaze[goal] = 'S'
    return goalMaze

In [18]:
def printMazePath(startState, goalState, path):
    """Prints out startState and goalState of the maze and shows the path to get there, if the path exists."""
    if path != 'failure' and path != 'cutoff':
        print('Solution path to the goal is {} steps long\n'.format(len(path)-1))
    print('Starting state:')
    printMaze(startState)
    print('Goal state:')
    printMaze(goalState)
    print('Solution path:')
    if path == 'failure':
        print('Failed: No solution found')
    elif path == 'cutoff':
        print('Path was cutoff')
    else:
        for i in path:
            printMaze(i)

In [19]:
def printMaze(maze):
    """Simply prints out the maze in it's 7x7 form"""
    for row in range(7):
        for col in range(7):
            print(maze[row*7 + col], end='')
        print()
    print()

#### Maze Testing

Again, for brevity, I'll only be showing an example of each case. Given the random generation, when run again, the notebook may not produce both a 'cutoff' and completed solution.

In [20]:
start = randomMaze()
goal = mazeGoal(start)
path = iterativeDeepeningSearch(start, goal, mazeActionsF, takeMazeActionF, 10)
printMazePath(start, goal, path)

Solution path to the goal is 7 steps long

Starting state:
*******
*S  ***
*    **
** ** *
*     *
*     *
*******

Goal state:
*******
*   ***
*    **
** ** *
*    S*
*     *
*******

Solution path:
*******
*S  ***
*    **
** ** *
*     *
*     *
*******

*******
* S ***
*    **
** ** *
*     *
*     *
*******

*******
*   ***
* S  **
** ** *
*     *
*     *
*******

*******
*   ***
*    **
**S** *
*     *
*     *
*******

*******
*   ***
*    **
** ** *
* S   *
*     *
*******

*******
*   ***
*    **
** ** *
*  S  *
*     *
*******

*******
*   ***
*    **
** ** *
*   S *
*     *
*******

*******
*   ***
*    **
** ** *
*    S*
*     *
*******



In [21]:
start = randomMaze()
goal = mazeGoal(start)
path = iterativeDeepeningSearch(start, goal, mazeActionsF, takeMazeActionF, 2)
printMazePath(start, goal, path)

Starting state:
*******
*   * *
*  *  *
***S **
*     *
* *   *
*******

Goal state:
*******
*   * *
*  *  *
***  **
*S    *
* *   *
*******

Solution path:
Path was cutoff


Above, I did not run a full check with `iterativeDeepeningSearch` to see if it handles failure correctly. I only tested it with printing out the puzzle's solution path. Now I'll give `iterativeDeepeningSearch` a maze that's impossible to solve to ensure it returns `failure` correctly.

In [22]:
start = ['*']*49 
goal  = ['*']*49
start[24] = 'S'
goal[8]   = 'S'
path = iterativeDeepeningSearch(start, goal, mazeActionsF, takeMazeActionF, 4)
printMazePath(start, goal, path)

Starting state:
*******
*******
*******
***S***
*******
*******
*******

Goal state:
*******
*S*****
*******
*******
*******
*******
*******

Solution path:
Failed: No solution found


#### General Size Maze

Since I know my maze generator works above, and my algorithm works with it, I'd like to generalize my maze to any `nxn` maze for fun. I'll make it so the user provides an 'n' to make the maze. The `nxn` will include the perimeter, which means the actual size of the random portion of the maze will be `(n-2)x(n-2)`.

In [23]:
def genMazeActionsF(state):
    """Takes the current nxn maze state and returns a list of the valid actions. Four possible actions: left, right, up, down."""
    actions = ['left','right','up','down']
    s = state.index('S')
    size = len(state)
    n = int(math.sqrt(size))
    if state[s-1] == '*':
        if 'left' in actions: actions.remove('left') # simple insurance check, even though actions shouldn't be removed before
    if state[s+1] == '*':
        if 'right' in actions: actions.remove('right')
    if state[s-n] == '*':
        if 'up' in actions: actions.remove('up')
    if state[s+n] == '*':
        if 'down' in actions: actions.remove('down')
    return actions

In [24]:
def takeGenMazeActionF(state, action):
    """Apply the given action to the nxn maze state and return the resulting state"""
    cp = copy.copy(state) # If a copy isn't taken, the action will permanently change the original state passed in, which is bad
    index = cp.index('S')
    size = len(state)
    n = int(math.sqrt(size))
    if(action == 'left'):
        temp = cp[index-1]
        cp[index-1] = 'S'
        cp[index] = temp
    elif(action == 'right'):
        temp = cp[index+1]
        cp[index+1] = 'S'
        cp[index] = temp
    elif(action == 'up'):
        temp = cp[index-n]
        cp[index-n] = 'S'
        cp[index] = temp
    else: #action is down
        temp = cp[index+n]
        cp[index+n] = 'S'
        cp[index] = temp
    return cp

In [25]:
def randomGenMaze(n):
    """Randomly generates a (n-2)x(n-2) maze with a starting position, 'S'. 
    The full size is nxn, but the entire perimeter is a wall"""
    if n < 3:
        print('n must be 3 or greater. Setting n to 3 for you.')
        n = 3
    maze = []
    space = ['*','*','*',' ',' ',' ',' ',' ',' ',' ','S'] # 3 walls, 7 spaces so walls aren't randomly picked as often. 1 start
    for i in range(n**2): # nxn maze, with walls on all edges; effectively a (n-2)x(n-2) randomly generated maze
        if i<n or i>(n*(n-1)-1) or i%n==0 or i%n==(n-1):
            maze.append('*')
        else:
            place = random.choice(space)
            if place == 'S':
                space.remove(place)
            if 'S' not in maze and 'S' in space and i == (n*(n-1)-2): # Forces start location if it wasn't randomly picked before
                maze.append('S')
                space.remove('S')
                continue
            maze.append(place)
    return maze

In [26]:
def printGenMazePath(startState, goalState, path):
    """Prints out startState and goalState of the nxn maze and shows the path to get there, if the path exists."""
    if path != 'failure' and path != 'cutoff':
        print('Solution path to the goal is {} steps long\n'.format(len(path)-1))
    print('Starting state:')
    printGenMaze(startState)
    print('Goal state:')
    printGenMaze(goalState)
    print('Solution path:')
    if path == 'failure':
        print('Failed: No solution found')
    elif path == 'cutoff':
        print('Path was cutoff')
    else:
        for i in path:
            printGenMaze(i)

In [27]:
def printGenMaze(maze):
    """Simply prints out the maze in it's nxn form"""
    size = len(maze)
    n = int(math.sqrt(size))
    for row in range(n):
        for col in range(n):
            print(maze[row*n + col], end='')
        print()
    print()

Might as well start generating mazes of the same size as the hard-coded generator above to see if it works.

In [28]:
start = randomGenMaze(7)
goal = mazeGoal(start)
path = iterativeDeepeningSearch(start, goal, genMazeActionsF, takeGenMazeActionF, 10)
printGenMazePath(start, goal, path)

Starting state:
*******
**S*  *
* *****
** *  *
*   * *
***  **
*******

Goal state:
*******
** *  *
* *****
** *  *
*   * *
*** S**
*******

Solution path:
Failed: No solution found


Let's make the maze a little larger this time.

In [29]:
start = randomGenMaze(20)
goal = mazeGoal(start)
path = iterativeDeepeningSearch(start, goal, genMazeActionsF, takeGenMazeActionF, 15)
printGenMazePath(start, goal, path)

Solution path to the goal is 9 steps long

Starting state:
********************
*  S     ** * * *  *
*        * *      **
* * *     ** *     *
*  ****  *   **    *
*  *    *   *  **  *
***        * *  *  *
*  * *    *      * *
** *  **  *  *    **
*   *              *
*     *   **       *
**     *       **  *
***    *  *    **  *
*  *     *      ** *
* *      *   * *   *
*      *        ****
***    *  * ***  * *
*  *     ** *     **
*    *  *   *     **
********************

Goal state:
********************
*        ** * * *  *
*        * *      **
* * *     ** *     *
*  ****  *   **    *
*  *  S *   *  **  *
***        * *  *  *
*  * *    *      * *
** *  **  *  *    **
*   *              *
*     *   **       *
**     *       **  *
***    *  *    **  *
*  *     *      ** *
* *      *   * *   *
*      *        ****
***    *  * ***  * *
*  *     ** *     **
*    *  *   *     **
********************

Solution path:
********************
*  S     ** * * *  *
*        * *      **
* * *    

Just for fun, I'd like to see what happens with a 100x100 (really 98x98) maze.

In [30]:
start = randomGenMaze(100)
goal = mazeGoal(start)
path = iterativeDeepeningSearch(start, goal, genMazeActionsF, takeGenMazeActionF, 10)
printGenMazePath(start, goal, path)

Starting state:
****************************************************************************************************
** S* * *     ** *   *   *       ** * * **** * *   ** **    **   **   * * *     * *    ** ** *    **
* * *    **    * *            ** *        *    * *             * *  * * * ** **  *    *            *
*  *      *** *  *  *    *        *    *     *        ***    *  *    *   *   *     *   * *      * **
*            *    *     *   ***     * **** ***  * *  *    *  *  *    **  ***  *   * * *  *       * *
** * *  *    *      * *   *   * *    **   *  *    **     *  *     ****         *    *     * ** *   *
* ** **  *     *   **      * *    *        *   *  *  *   *     *   *          *      **    *     * *
* *  **  *  *  *  *  *  *  * *     **       *              *      * *  *  *   *    * **  **   *    *
*       *    **     *   *    **  **    *   * *      ***    * **     ** **   *  *** * **   **   **  *
**        *    *** *    *  *  *   *      *   *   ***    *    *  *      *   

*  *   *    * *   *    *       *    * **  *  ****        *  ** * ***    * *    * **   **    * *  * *
**  *  ** *    **      **      **   **    *   * *      **  ***   * * ** *    **  *    *     **     *
* **  * * * *   *        *    *       *      ***    * *     ** *        **      * *    *   *   **  *
* *     *  **   * * **      * *   *   *     *   *           *     ***    ** *  *        *  *   *** *
*   *    * *  * *  * *  *****           ***  *  *** * *  * *          **  * * *     ** * **  ***  **
*      *   **  *** *  ***  *          *     **        *    *  *   **  * *      **  **    **** ** * *
*     ** ** *  * *  ** *   *    ****        **         **     *    * **  *   *   *  *   ** **  ** **
*    *   ***  **    * *  *      *      *   *         **    ***  ** *    *          *  * *  * *     *
* *        *         * ***     * *  *  *         *****  *     **   *    *     ** *  * * *    **  ***
*******************************************************************************************

*  * *        *  *    * ** *    *    *                *   *    *    *    *  ** *    **    **       *
**** ***      *   *  * * **  *  *      *   *        * *  * *****   *  *  ***  *  ****         **** *
*    **  ***     * * *   *         **       *  ** ***   *   *     *  *** *  *    ***    **  * **   *
*  *  *    * * *   *      * * * *               ** ****             ** ** *           ** *    *    *
*** *     ** ***    * *           *   *    **    ** ** ***       **  * *       **  ***      *   * **
**   *** ***    *  **     * *  ***   *       * **   *                  * *     * **               **
*  **  ****   * *    *         *     *        *  *           *    *     * *    *  *** *** *    *  **
* **     ***        * *    ** ***     *    *  ***        *      * **    * **  **   ** **     *   ***
**       **   *  * ***  *  * *****          ***  *         *     * *      *     *         * *      *
*  * *         *    * *   * * *                ** *       * *   * *  **      * *  ***  *** 

There may be too many walls. I set the random selection so it would pick roughly 30% walls and 70% open spaces. It might be worth tinkering with in the future to see what percentage of walls may be ideal to be able to randomly generate solvable mazes most of the time. This is admittedly a much more difficult issue to tackle once mazes get larger and larger. It would likely need more checks and balances on the random generation that my simple maze generator isn't currently built to do.

It is also difficult to completely test success on this maze. Given the size of the maze, a potentially solvable maze could take many steps to actually find. This would take a substantial amount of time, so I'm capping off `maxDepth` at 10 so it runs relatively fast.

## Grading

In [31]:
%run -i A2grader.py


Searching this graph:
 {'a': ['b', 'z', 'd'], 'b': ['a'], 'y': ['z'], 'e': ['z'], 'd': ['y']}

Looking for path from a to y with max depth of 1.
 5/ 5 points. Your search correctly returned cutoff

Looking for path from a to y with max depth of 5.
10/10 points. Your search correctly returned ['a', 'z']

Testing findBlank_8p([1, 2, 3, 4, 5, 6, 7, 0, 8])
 5/ 5 points. Your findBlank_8p correctly returned 2 1

Testing actionsF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8])
10/10 points. Your actionsF_8p correctly returned ['left', 'right', 'up']

Testing takeActionF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8],up)
10/10 points. Your takeActionsF_8p correctly returned [1, 2, 3, 4, 0, 6, 7, 5, 8]

Testing iterativeDeepeningSearch([1, 2, 3, 4, 5, 6, 7, 0, 8],[0, 2, 3, 1, 4,  6, 7, 5, 8], actionsF_8p, takeActionF_8p, 5)
20/20 points. Your search correctly returned [[1, 2, 3, 4, 5, 6, 7, 0, 8], [1, 2, 3, 4, 0, 6, 7, 5, 8], [1, 2, 3, 0, 4, 6, 7, 5, 8], [0, 2, 3, 1, 4, 6, 7, 5, 8]]

Testing iterativeDeepeningSearch([5, 2, 